### Import packages

In [426]:
#import the packages
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

### Define constants

In [427]:
#define constants
base_url = 'https://api.foursquare.com/v2'
client_id = ''
client_secret = ''
wikiUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
V='20180604'

### Fetch Wikipedia article

In [428]:
#Fetch Wikipedia article
resp = requests.get(wikiUrl)
html_text = resp.text

### Parse and build data frame

In [429]:
#Parse and build data frame
soup = BeautifulSoup(html_text, 'html.parser')
postal_codeTable = soup.find('table')
rows = postal_codeTable.find_all('tr')
header = []
for th in rows[0].find_all('th'):
    header.append(th.text.replace('\n',''))
df_postalcode = pd.DataFrame(columns = header)
rows = rows[1:]
#print(rows)
rownum  = 0
for row in rows:
    df_row = []
    for td in row.find_all('td'):
        df_row.append(td.text.replace('\n',''))
    #print(df_row)
    df_postalcode.loc[rownum] = df_row
    rownum = rownum + 1
df_postalcode.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data clean up per instruction -
+  Remove those where borough is 'Not assigned'
+  Group the neighbourhoods
+  Replace Neighbourhood with Borough when Neighbourhood is not assigned

In [430]:
#filter out those where borough is 'Not assigned'
df_postalcode = df_postalcode[df_postalcode['Borough'] != 'Not assigned'].reset_index()
df_postalcode.drop('index', axis=1,inplace=True)

#Group the neighbourhoods
df_postalcode = pd.DataFrame(df_postalcode.groupby(['Postcode','Borough'],axis=0)
                             .apply(lambda x: ','.join(x['Neighbourhood'].values)).reset_index())

df_postalcode.columns = ['Postcode','Borough','Neighbourhood']

#Replace Neighbourhood with Borough when Neighbourhood is not assigned
df_postalcode['Neighbourhood'].where(cond=df_postalcode['Neighbourhood'] != 'Not assigned', other=df_postalcode['Borough'] 
                    ,inplace = True
                    ,axis=0)


### Shape of the final DataFrame

In [431]:
df_postalcode.shape

(103, 3)

In [432]:
df_postalcode.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Part 2 - Merge longitude, latitude with df_postalcode dataframe

In [433]:
df_longlat = pd.read_csv('http://cocl.us/Geospatial_data')
df_longlat.rename(columns={'Postal Code':'Postcode'}, inplace = True)
df_postalcode = pd.merge(df_postalcode,df_longlat, on='Postcode',how='inner')
df_postalcode.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3  - Exploration and Analysis of Toronto Boroughs' data foursquare API data.
For exploring and clustering neighbourhoods of Toronto I will be doing following steps. Its very similar to the New York Lab Example in the course, but instead of one borough I will be analyzing all the given boroughs and neighbourhood
+ Create a map of Toronto with neighborhoods superimposed on top.
+ For each neighbourhood get top 100 venues using foursquare within 500 meters range. 
        + For the venues extracted extract long, lat, name and category
+ Repeat the steps above for all the neighbourhood and join data with original dataframe.


### Map of Toronto with neighbourhoods superimposed.

In [434]:
# Get log/lat of Toronto
address_Toronto = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address_Toronto)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [435]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_postalcode['Latitude'], df_postalcode['Longitude'], df_postalcode['Borough'], df_postalcode['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### First Neighborhood in dataframe df_postalcode. This will eventually be repeated for all neighborhoods

In [436]:
neighborhood_latitude = df_postalcode.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_postalcode.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_postalcode.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [437]:
url =  '{}/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit=100&radius=5000'.format(base_url,client_id,client_secret,neighborhood_latitude,neighborhood_longitude, V)
print(url)



https://api.foursquare.com/v2/venues/explore?client_id=4YYUY0FWACWQSMRZS1SWOQKU4J5T23CFDEHFDJUZGAJBJQL2&client_secret=J2MKJZVR4HMIPD2YXI1VPVNQAQN4012GJTJKGQOJ31YBK240&ll=43.806686299999996,-79.19435340000001&v=20180604&limit=100&radius=5000


In [438]:
results = requests.get(url).json()

In [439]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [440]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Toronto Zoo,Zoo,43.820582,-79.181551
3,Polar Bear Exhibit,Zoo,43.823372,-79.185145
4,Australasia Pavillion,Zoo Exhibit,43.822563,-79.183286


### Repeat for all neighbourhoods of Toronto
Create the function similar to New York Lab to (getNearbyVenues)

In [441]:
LIMIT =100
radius = 5000
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            V, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [442]:
toronto_venues = getNearbyVenues(names=df_postalcode['Neighbourhood'],
                                   latitudes=df_postalcode['Latitude'],
                                   longitudes=df_postalcode['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beac

In [443]:
toronto_venues.shape

(2259, 7)

In [444]:
toronto_venues.shape

(2259, 7)

In [445]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'], axis=1, inplace=True)
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape


(2259, 282)

In [446]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [447]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Agincourt----
               venue  freq
0       Skating Rink  0.25
1     Breakfast Spot  0.25
2             Lounge  0.25
3     Clothing Store  0.25
4  Accessories Store  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
               venue  freq
0               Park  0.67
1         Playground  0.33
2  Mobile Phone Shop  0.00
3      Moving Target  0.00
4      Movie Theater  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0           Pizza Place   0.2
1         Grocery Store   0.2
2  Fast Food Restaurant   0.1
3   Fried Chicken Joint   0.1
4            Beer Store   0.1


----Alderwood,Long Branch----
                venue  freq
0         Pizza Place  0.18
1     

#### Most Common Venues function

In [448]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Most common 10 venues for each neighborhood

In [449]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Hotel,Bakery,Restaurant,Gym,Clothing Store
1,Agincourt,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Grocery Store,Coffee Shop,Sandwich Place,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Pharmacy,Greek Restaurant,Event Space
4,"Alderwood,Long Branch",Pizza Place,Dance Studio,Pharmacy,Gym,Sandwich Place,Athletics & Sports,Skating Rink,Coffee Shop,Pool,Pub


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [450]:
#toronto_grouped.head()
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering.head()
# # run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# # check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 2, 2, 3, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [451]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()
df_postalcode.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
toronto_merged = df_postalcode

# # merge toronto_grouped with df_postalcode to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Moving Target,Bar,Fish & Chips Shop,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2.0,Pizza Place,Breakfast Spot,Rental Car Location,Electronics Store,Mexican Restaurant,Medical Center,Ethiopian Restaurant,Empanada Restaurant,Event Space,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Insurance Office,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Athletics & Sports,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Doner Restaurant,Donut Shop,Drugstore


### Plot on Map various neighbourhood with color according to their cluster

In [452]:
map_toronto1 = folium.Map(location=[latitude, longitude], zoom_start=10)
color_labels = ['blue','green','yellow','orange','pink']
# add markers to map
for lat, lng, borough, neighborhood,clusterlabel in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']):
    label = '{}, {} - {}'.format(neighborhood, borough, str(clusterlabel))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= 'black' if math.isnan(clusterlabel) else color_labels[int(clusterlabel)],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto1)  
    
map_toronto1